In [397]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime
import folium
import time
from matplotlib import colormaps
from matplotlib.colors import rgb2hex
import scrapy
from scrapy import Selector
'''
import plotly.express as px
from scrapy.crawler import CrawlerProcess
import csv
'''


'\nimport plotly.express as px\nfrom scrapy.crawler import CrawlerProcess\nimport csv\n'

In [349]:
load_dotenv()

Open_Weather_API_Key_Myriam= os.getenv("Open_Weather_API_Key_Myriam") 

# Extraction des informations géographiques des 35 plus belles villes

Sur la liste des 35 plus belles villes, nous avons 34 noms de villes + 1 lieu d'interêt qui n'est pas une ville (Gorges du Verdon). Pour récupérer les latitudes et longitudes des 34 villes nous pouvons facilement utiliser l'API openweathermap. Cette API permet facilement de collecter également le pays et la région, ce qui est utile pour contrôler d'un même coût les éventuelles erreures liées à des villes homonymes dans le monde.
Cependant, cette API récupère les informations en se basant sur un nom de ville. Elle ne reconnait pas les Gorges du Verdon. Pour palier ce problème, une seconde API a été utilisée, nominatim.openstreetmap. Le nombre de requete est plus limité et elle ne permet pas de récupérer la région et le pays. C'est en visualisant les villes sur une carte qu'on pourra confirmer la bonne géolocalisation.

In [350]:
best_cities= ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle",
#"Gorges du Verdon" exclu de la liste dans un premier temps car n'est pas une ville mais un point d'intérêt.
]


In [351]:
best_cities[-1]

'La Rochelle'

In [352]:
#création d'un dataframe avec la liste des best cities
df = pd.DataFrame(data = best_cities, columns = ["cities"])
df

,cities
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen
5,Paris
6,Amiens
7,Lille
8,Strasbourg
9,Chateau du Haut Koenigsbourg


## Import des longitudes & latitudes des 35 best cities

In [353]:
df.shape

(34, 1)

In [354]:
for i,best_city in enumerate(best_cities):
    r = requests.get(f"http://api.openweathermap.org/geo/1.0/direct?q={best_city}&limit=1&appid={Open_Weather_API_Key_Myriam}")
    if r.status_code == 200:
        data = r.json()
  
        df.loc[i,"name"] = data[0]["name"] #pour s'assurer de la bonne attribution de la ville
        df.loc[i,"country"] = data[0]["country"] #pour vérifier la localisation
        df.loc[i,"state"] = data[0]["state"] # pour vérifier la localisation
        df.loc[i,"lat"] = data[0]["lat"]
        df.loc[i,"lon"] = data[0]["lon"]
    
    else:
        print(f"Erreur lors de la requête pour {best_cities} :", r.status_code)

df

,cities,name,country,state,lat,lon
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.511460
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966
5,Paris,Paris,FR,Ile-de-France,48.858890,2.320041
6,Amiens,Amiens,FR,Hauts-de-France,49.894171,2.295695
7,Lille,Lille,FR,Hauts-de-France,50.636565,3.063528
8,Strasbourg,Strasbourg,FR,Grand Est,48.584614,7.750713
9,Chateau du Haut Koenigsbourg,Château du Haut-Kœnigsbourg,FR,Grand Est,48.249523,7.345492


In [355]:
url = "https://nominatim.openstreetmap.org/search"

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'    }

lieux = ["Gorges du Verdon"]

for lieu in lieux:
    params = {
        'q': lieu,
        'format': 'json',
        'limit': 1
    }

response = requests.get(url, params=params, headers=headers)
response

<Response [200]>

In [356]:
# pour trouver les coordonnés des Gorges du Verdon, utiisation d'une seconde API (Elle est plus restrictive en termes du nombre de requete, c'est pour celà que la préédente API a été favorisée)
def rechercher_coordonnees(lieux):
    url = "https://nominatim.openstreetmap.org/search"
    coordonnees = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'    }

    for lieu in lieux:
        params = {
            'q': lieu,
            'format': 'json',
            'limit': 1
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            if data:
                address = data[0].get('address', {})
                coordonnees.append({
                    'cities': lieu,
                    'name': address.get('city'),
                    'country': address.get('country'), #information non trouvée avec l'API
                    'state': address.get('state'), #information non trouvée avec l'API
                    'lat': data[0].get('lat'),
                    'lon': data[0].get('lon'),
                })
            else:
                coordonnees.append({
                    'cities': lieu,
                    'name': None,
                    'country': None,
                    'state': None,
                    'lat': None,
                    'lon': None

                })
        else:
            print(f"Erreur lors de la requête pour {lieu} :", response.status_code)
            coordonnees.append({
                    'cities': lieu,
                    'name': None,
                    'country': None,
                    'state': None,
                    'lat': None,
                    'lon': None,
            })

        # Attendre un peu entre les requêtes pour éviter de dépasser les limites
        time.sleep(1)

    return coordonnees


coordonnees = rechercher_coordonnees(["Gorges du Verdon"])
print(coordonnees)


[{'cities': 'Gorges du Verdon', 'name': None, 'country': None, 'state': None, 'lat': '43.7496562', 'lon': '6.3285616'}]


In [357]:
#réalisation d'un dataframe avec le même colonage que le dataframe initial
gdv = pd.DataFrame(coordonnees)
gdv

,cities,name,country,state,lat,lon
0,Gorges du Verdon,None,None,None,43.7496562,6.3285616


In [358]:
#Fusion des deux dataframes
df_merged = pd.concat([df, gdv], ignore_index=True)
df_merged

,cities,name,country,state,lat,lon
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.51146
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966
5,Paris,Paris,FR,Ile-de-France,48.85889,2.320041
6,Amiens,Amiens,FR,Hauts-de-France,49.894171,2.295695
7,Lille,Lille,FR,Hauts-de-France,50.636565,3.063528
8,Strasbourg,Strasbourg,FR,Grand Est,48.584614,7.750713
9,Chateau du Haut Koenigsbourg,Château du Haut-Kœnigsbourg,FR,Grand Est,48.249523,7.345492


In [359]:
# Création d'une carte pour visualiser les 35 best cities/points d'intérêts et leur localisation exacte
map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Définir le marker 
icon_url = "https://images.icon-icons.com/3565/PNG/512/tree_flower_plant_nature_landscape_icon_225325.png"

# Ajouter les villes à la carte
for i, row in df_merged.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['cities'],
        icon=folium.CustomIcon(icon_url, icon_size=(40, 40))  # Ajustez la taille de l'icône si nécessaire
    ).add_to(map)

# Sauvegarder la carte dans un fichier HTML
map.save('maps/map_best_city.html')

# Afficher la carte
map

In [366]:
df_merged = df_merged.drop("name", axis=1)

In [367]:
df_merged.head()

,cities,country,state,lat,lon
0,Mont Saint Michel,FR,Normandy,48.635954,-1.51146
1,St Malo,FR,Brittany,48.649518,-2.026041
2,Bayeux,FR,Normandy,49.276462,-0.702474
3,Le Havre,FR,Normandy,49.493898,0.107973
4,Rouen,FR,Normandy,49.440459,1.093966


In [368]:
# Enregistrement du DataFrame fusionné en fichier CSV
df_merged.to_csv('data/35_best_cities_latlon.csv', index=False)

## Import des informations météo

In [369]:
df=pd.read_csv("data/35_best_cities_latlon.csv")

In [370]:
url = f"https://api.openweathermap.org/data/3.0/onecall?lat={df.loc[0, 'lat']}&lon={df.loc[0, 'lon']}&units=metric&exclude=hourly,daily&appid={Open_Weather_API_Key_Myriam}"

r = requests.get(url)
data = r.json()
r

<Response [200]>

In [371]:
# import des informations météorologiques importantes
for i in df.index:
    lat = df.loc[i, 'lat']
    lon = df.loc[i, 'lon']

    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&exclude=hourly,daily&appid={Open_Weather_API_Key_Myriam}"

    r = requests.get(url)

    if r.status_code == 200:
        data = r.json()
  
        df.loc[i,"date"] = data["current"]["dt"]
        df.loc[i, "temp"] = data["current"]["temp"]
        df.loc[i, "feels_like"] = data["current"]["feels_like"]
        df.loc[i, "uvi"] = data["current"]["uvi"]
        df.loc[i, "humidity"] = data["current"]["humidity"]
        df.loc[i, "wind_speed"] = data["current"]["wind_speed"]
        df.loc[i, "description"] = data["current"]["weather"][0]["description"]
        
    else:
        print(f"Erreur lors de la requête pour l'index {i} : {r.status_code}")
        print(r.text)  # Imprime le message d'erreur pour plus de détails



In [372]:
df.head()

,cities,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Mont Saint Michel,FR,Normandy,48.635954,-1.511460,1.745499e+09,13.95,13.23,4.75,70.0,3.42,broken clouds
1,St Malo,FR,Brittany,48.649518,-2.026041,1.745499e+09,15.29,14.62,5.41,67.0,3.09,broken clouds
2,Bayeux,FR,Normandy,49.276462,-0.702474,1.745499e+09,12.14,11.37,4.33,75.0,4.45,broken clouds
3,Le Havre,FR,Normandy,49.493898,0.107973,1.745499e+09,11.51,10.99,4.59,87.0,4.63,overcast clouds
4,Rouen,FR,Normandy,49.440459,1.093966,1.745499e+09,13.66,13.01,3.15,74.0,3.60,broken clouds


In [373]:
#conversion de la colonne date en formation datetime
df['date'] = pd.to_datetime(df['date'], unit='s')
df.head()

,cities,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Mont Saint Michel,FR,Normandy,48.635954,-1.511460,2025-04-24 12:50:01,13.95,13.23,4.75,70.0,3.42,broken clouds
1,St Malo,FR,Brittany,48.649518,-2.026041,2025-04-24 12:50:05,15.29,14.62,5.41,67.0,3.09,broken clouds
2,Bayeux,FR,Normandy,49.276462,-0.702474,2025-04-24 12:50:05,12.14,11.37,4.33,75.0,4.45,broken clouds
3,Le Havre,FR,Normandy,49.493898,0.107973,2025-04-24 12:50:05,11.51,10.99,4.59,87.0,4.63,overcast clouds
4,Rouen,FR,Normandy,49.440459,1.093966,2025-04-24 12:50:05,13.66,13.01,3.15,74.0,3.60,broken clouds


In [374]:
# Enregistrer le DataFrame en fichier CSV
df.to_csv('data/35_best_cities_weather.csv', index=False)

# Rapide EDA 

In [375]:
bc = pd.read_csv("data/35_best_cities_weather.csv")
bc

,cities,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Mont Saint Michel,FR,Normandy,48.635954,-1.511460,2025-04-24 12:50:01,13.95,13.23,4.75,70.0,3.42,broken clouds
1,St Malo,FR,Brittany,48.649518,-2.026041,2025-04-24 12:50:05,15.29,14.62,5.41,67.0,3.09,broken clouds
2,Bayeux,FR,Normandy,49.276462,-0.702474,2025-04-24 12:50:05,12.14,11.37,4.33,75.0,4.45,broken clouds
3,Le Havre,FR,Normandy,49.493898,0.107973,2025-04-24 12:50:05,11.51,10.99,4.59,87.0,4.63,overcast clouds
4,Rouen,FR,Normandy,49.440459,1.093966,2025-04-24 12:50:05,13.66,13.01,3.15,74.0,3.60,broken clouds
5,Paris,FR,Ile-de-France,48.858890,2.320041,2025-04-24 12:50:05,15.21,14.59,3.04,69.0,4.63,overcast clouds
6,Amiens,FR,Hauts-de-France,49.894171,2.295695,2025-04-24 12:50:05,12.88,12.21,2.61,76.0,4.59,overcast clouds
7,Lille,FR,Hauts-de-France,50.636565,3.063528,2025-04-24 12:50:05,12.98,12.08,4.78,67.0,6.69,overcast clouds
8,Strasbourg,FR,Grand Est,48.584614,7.750713,2025-04-24 12:50:05,13.00,12.34,3.68,76.0,2.57,light rain
9,Chateau du Haut Koenigsbourg,FR,Grand Est,48.249523,7.345492,2025-04-24 12:50:05,10.90,9.80,2.49,67.0,3.09,broken clouds


In [376]:
bc.shape

(35, 12)

In [377]:
df_sorted = bc.sort_values(by='temp', ascending=True)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='temp', y='cities', orientation='h',
             title='Températures par Ville',
             labels={'temp': 'Température (°C)', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [378]:
df_sorted = bc.sort_values(by='feels_like', ascending=True)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='feels_like', y='cities', orientation='h',
             title='Températures ressenties par Ville',
             labels={'feels_like': 'Température ressentie (°C)', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [379]:
df_sorted = bc.sort_values(by='uvi', ascending=True)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='uvi', y='cities', orientation='h',
             title='indices UV par Ville',
             labels={'uvi': 'Indice UV', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [380]:
df_sorted = bc.sort_values(by='humidity', ascending=False)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='humidity', y='cities', orientation='h',
             title='Humidité par Ville',
             labels={'humidity': 'Humidité', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [381]:
df_sorted = bc.sort_values(by='wind_speed', ascending=False)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='wind_speed', y='cities', orientation='h',
             title='Vitesse du vent par Ville',
             labels={'wind_speed': 'wind_speed', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [389]:
# Dictionnaire pour mapper les descriptions aux icônes
icon_mapping = {
    'clear sky': "https://images.icon-icons.com/2791/PNG/512/sunny_day_weather_sun_icon_177555.png",  
    'scattered clouds': 'https://images.icon-icons.com/2791/PNG/512/partly_cloudy_day_sun_clouds_weather_icon_177560.png',
    'few clouds' : 'https://images.icon-icons.com/857/PNG/512/weather-2_icon-icons.com_67742.png',
    'broken clouds': 'https://images.icon-icons.com/2791/PNG/512/cloudy_cloud_weather_icon_177580.png',
    'overcast clouds': 'https://images.icon-icons.com/2791/PNG/512/overcast_cloud_weather_icon_177561.png',
    'light intensity drizzle rain' : 'https://images.icon-icons.com/139/PNG/512/drizzle_day_weather_20907.png',
    'light intensity drizzle': 'https://images.icon-icons.com/139/PNG/512/drizzle_day_weather_20907.png',
    'mist': 'https://images.icon-icons.com/2791/PNG/512/fog_weather_icon_177578.png',
    'light rain' : 'https://images.icon-icons.com/2791/PNG/512/rain_cloud_weather_icon_177568.png',
    'heavy rain' : 'https://images.icon-icons.com/2791/PNG/512/heavy_rain_weather_icon_177574.png',
    'heavy intensity rain' : "https://images.icon-icons.com/2791/PNG/512/heavy_rain_day_weather_icon_177573.png",
    'moderate rain' : 'https://images.icon-icons.com/2791/PNG/512/heavy_rain_day_weather_icon_177573.png',
    'thunder storm' : 'https://images.icon-icons.com/2791/PNG/512/thunder_thunderbolt_cloud_weather_icon_177554.png',
    'snow' : 'https://images.icon-icons.com/2791/PNG/512/heavy_snow_weather_icon_177571.png'
}

# Créer une carte centrée sur la France
map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter les villes à la carte avec des icônes personnalisées
for i, row in bc.iterrows():
    icon_url = icon_mapping.get(row['description'], 'https://images.icon-icons.com/1744/PNG/512/3643775-caution-exclamation-mark-sign-triangle_113444.png')  #si non trouvé dans le dico, ce logo apparait
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['cities'],
        icon=folium.CustomIcon(icon_url, icon_size=(40, 40))  # Ajustez la taille de l'icône si nécessaire
    ).add_to(map)

# Sauvegarder la carte dans un fichier HTML
map.save('maps/map_meteo_35_best_city.html')

# Afficher la carte
map


# Selection des meilleures destinations

Nous sommes au printemps. Nous cherchons une distination qui nous changerait de la grisaille parisienne. Pour définir les villes avec la meilleure météo, nous pouvons définir les conditions suivantes :
- ciel dégagé
- hautes température rééel et ressentie
- Indice UV élevé
- faible taux d'humidité
- faible vitesse du vent

In [384]:
# Filtrer les villes avec la description "clear sky"
filtered_bc = bc[bc['description'] == 'clear sky']

# Sélectionner les 5 meilleures villes pour chaque critère parmi les villes filtrées.
top_feels_like = set(filtered_bc.nlargest(5, 'feels_like')['cities'])# critère le plus important, donc le plus restrictif
#top_temp = set(filtered_bc.nlargest(10, 'temp')['cities']) #critère retiré car redondant avec la précédent
top_uvi = set(filtered_bc.nlargest(10, 'uvi')['cities'])
top_humidity = set(filtered_bc.nsmallest(10, 'humidity')['cities'])
top_wind_speed = set(filtered_bc.nsmallest(15, 'wind_speed')['cities'])

# Trouver l'intersection des ensembles
best_weather = top_feels_like &  top_uvi & top_humidity & top_wind_speed
#top_temp &

# Afficher la liste des meilleures villes
print("Villes avec la meilleure météo :")
print(best_weather)

Villes avec la meilleure météo :
{'Bormes les Mimosas', 'Marseille', 'Aigues Mortes', 'Nimes', 'Cassis'}


In [385]:
best_weather = list(best_weather) #transformer le set en liste
best_weather_df = bc[bc['cities'].isin(best_weather)] #puis filter le dataframe sur la liste
best_weather_df = best_weather_df.sort_values(by='feels_like', ascending=False)
best_weather_df


,cities,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
17,Bormes les Mimosas,FR,Provence-Alpes-Côte d'Azur,43.150697,6.341928,2025-04-24 12:50:06,22.24,21.54,6.05,39.0,5.66,clear sky
24,Aigues Mortes,FR,Occitania,43.565823,4.191284,2025-04-24 12:50:07,21.87,21.16,6.10,40.0,6.69,clear sky
23,Nimes,FR,Occitania,43.837425,4.360069,2025-04-24 12:50:07,21.22,20.44,6.10,40.0,7.72,clear sky
19,Marseille,FR,Provence-Alpes-Côte d'Azur,43.296174,5.369953,2025-04-24 12:50:06,20.30,19.27,6.03,34.0,13.89,clear sky
18,Cassis,FR,Provence-Alpes-Côte d'Azur,43.214036,5.539632,2025-04-24 12:50:06,20.17,19.13,6.08,34.0,13.70,clear sky


In [386]:
best_weather_df["cities"]

17    Bormes les Mimosas
24         Aigues Mortes
23                 Nimes
19             Marseille
18                Cassis
Name: cities, dtype: object

In [387]:
# Enregistrer le DataFrame en fichier CSV
best_weather_df.to_csv('data/top_best_cities_weather.csv', index=False)

In [392]:
best_weather_df = pd.read_csv("data/top_best_cities_weather.csv")
best_weather_df

,cities,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Bormes les Mimosas,FR,Provence-Alpes-Côte d'Azur,43.150697,6.341928,2025-04-24 12:50:06,22.24,21.54,6.05,39.0,5.66,clear sky
1,Aigues Mortes,FR,Occitania,43.565823,4.191284,2025-04-24 12:50:07,21.87,21.16,6.10,40.0,6.69,clear sky
2,Nimes,FR,Occitania,43.837425,4.360069,2025-04-24 12:50:07,21.22,20.44,6.10,40.0,7.72,clear sky
3,Marseille,FR,Provence-Alpes-Côte d'Azur,43.296174,5.369953,2025-04-24 12:50:06,20.30,19.27,6.03,34.0,13.89,clear sky
4,Cassis,FR,Provence-Alpes-Côte d'Azur,43.214036,5.539632,2025-04-24 12:50:06,20.17,19.13,6.08,34.0,13.70,clear sky


In [393]:
# visualiser le top5 des destinations selectionnées
map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Définir le marker
icon_url = "https://images.icon-icons.com/3565/PNG/512/tree_flower_plant_nature_landscape_icon_225325.png"

# Ajouter les villes à la carte avec des icônes personnalisées
for i, row in best_weather_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['cities'],
        icon=folium.CustomIcon(icon_url, icon_size=(40, 40))  # Ajustez la taille de l'icône si nécessaire
    ).add_to(map)

# Sauvegarder la carte dans un fichier HTML
map.save('maps/map_top_best_destinations.html')

# Afficher la carte
map


# Scraping Booking

In [394]:
best_weather_df

,cities,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Bormes les Mimosas,FR,Provence-Alpes-Côte d'Azur,43.150697,6.341928,2025-04-24 12:50:06,22.24,21.54,6.05,39.0,5.66,clear sky
1,Aigues Mortes,FR,Occitania,43.565823,4.191284,2025-04-24 12:50:07,21.87,21.16,6.10,40.0,6.69,clear sky
2,Nimes,FR,Occitania,43.837425,4.360069,2025-04-24 12:50:07,21.22,20.44,6.10,40.0,7.72,clear sky
3,Marseille,FR,Provence-Alpes-Côte d'Azur,43.296174,5.369953,2025-04-24 12:50:06,20.30,19.27,6.03,34.0,13.89,clear sky
4,Cassis,FR,Provence-Alpes-Côte d'Azur,43.214036,5.539632,2025-04-24 12:50:06,20.17,19.13,6.08,34.0,13.70,clear sky


## Test des selecteurs pour extraction des urls

In [395]:
best_cities_list=best_weather_df['cities'].tolist()
best_cities_list

['Bormes les Mimosas', 'Aigues Mortes', 'Nimes', 'Marseille', 'Cassis']

In [396]:
import urllib.parse
urls= ['https://www.booking.com/searchresults.html?ss=' + urllib.parse.quote(city) for city in best_cities_list]
urls

['https://www.booking.com/searchresults.html?ss=Bormes%20les%20Mimosas',
 'https://www.booking.com/searchresults.html?ss=Aigues%20Mortes',
 'https://www.booking.com/searchresults.html?ss=Nimes',
 'https://www.booking.com/searchresults.html?ss=Marseille',
 'https://www.booking.com/searchresults.html?ss=Cassis']

In [398]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
url = urls[0]
r = requests.get(url, headers=headers)

response = Selector(text=r.text)

In [399]:
r

<Response [200]>

In [400]:
hotel_name=response.css("div.f6431b446c.a15b38c233 ::text").getall()
hotel_name

['Cosy studio - Place du Figuier',
 'Les Terrasses de Lô',
 'Bord de mer, charmant studio entièrement équipé',
 'Superbe T2 4 étoiles climatisé Favière refait à neuf',
 'Boulevard du Soleil',
 'Studio Liège, vue mer au village de Bormes les Mimosas',
 'La Lavandière',
 'Cosy 36m With Nice Sea View Center Village !',
 'Hotel La Voile',
 'Hôtel Paradis',
 'Studio idéal séjour en amoureux rez de jardin proche plage',
 'Les deux mas',
 'Hostellerie du Cigalou',
 'Le petit atelier',
 "L'Amirauté 150m de la plage",
 'Hotel Restaurant Bellevue',
 'Hotel Les Jardins de Bormes, sauna & jacuzzi',
 'Coeur de village',
 'Studio cosy avec vue sur la mer à Bormes-les-Mimosas, 20 m².',
 'Villa avec piscine - confort et tranquillité',
 'Villa Merveille',
 "SELECT'soHOME - T2 proche de la station balnéaire de La Favière avec parking privé ! - AZUR-17",
 'Gaou Benat, village des fourches, Label patrimoine XXe siècle',
 'La Recampado',
 'Hôtel de la Plage - HDLP']

In [401]:
hotel_url = response.css("a.a78ca197d0 ::attr(href)").getall()
hotel_url

['https://www.booking.com/hotel/fr/studio-figuier-bormes-les-mimosas.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGXNlYXJjaF9ib3JtZXMgbGVzIG1pbW9zYXNIM1gEaE2IAQGYAQm4ARfIAQzYAQHoAQH4AQOIAgGoAgO4Av7qqMAGwAIB0gIkMzc3NWY3MTUtOGVlOS00MGEzLThmNGEtYzBjNTdlOWM2ZmMy2AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=b5fd5b3f08ec02ed&srepoch=1745499519&from=searchresults',
 'https://www.booking.com/hotel/fr/les-terrasses-de-lo.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGXNlYXJjaF9ib3JtZXMgbGVzIG1pbW9zYXNIM1gEaE2IAQGYAQm4ARfIAQzYAQHoAQH4AQOIAgGoAgO4Av7qqMAGwAIB0gIkMzc3NWY3MTUtOGVlOS00MGEzLThmNGEtYzBjNTdlOWM2ZmMy2AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=b5fd5b3f08ec02ed&srepoch=1745499519&from=searchresults',
 'https://www.booking.com/hotel/fr/bord-de-mer-charmant-studio-entierement-equipe.en-gb.html?aid=304142&

## Test des selecteurs pour extractions des infos par hotel

In [402]:
#étape réalisée après avoir scrapé les urls d'hotel

In [403]:
#extraction des URL d'hotels
with open('data/booking_hotels.json', 'r', encoding="utf-8") as file:
    data = json.load(file)
hotel_urls = [item['hotel_url'] for item in data]
hotel_urls

['https://www.booking.com/hotel/fr/lepregaloffrenimes.fr.html?aid=304142&label=gen173nr-1FCAQoggJCDHNlYXJjaF9uaW1lc0gzWARoTYgBAZgBDbgBF8gBDNgBAegBAfgBA4gCAagCA7gCl-yowAbAAgHSAiQ1OGUyMGZlOS01MjdjLTQ2OGItYjgyZC1jMWU0OGY2NGQ4MjDYAgXgAgE&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=25&hapos=25&sr_order=popularity&srpvid=d5965b8b343f0269&srepoch=1745499672&from=searchresults',
 'https://www.booking.com/hotel/fr/mezza-nimes.fr.html?aid=304142&label=gen173nr-1FCAQoggJCDHNlYXJjaF9uaW1lc0gzWARoTYgBAZgBDbgBF8gBDNgBAegBAfgBA4gCAagCA7gCl-yowAbAAgHSAiQ1OGUyMGZlOS01MjdjLTQ2OGItYjgyZC1jMWU0OGY2NGQ4MjDYAgXgAgE&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=21&hapos=21&sr_order=popularity&srpvid=d5965b8b343f0269&srepoch=1745499672&from=searchresults',
 'https://www.booking.com/hotel/fr/nimotel.fr.html?aid=304142&label=gen173nr-1FCAQoggJCDHNlYXJjaF9uaW1lc0gzWARoTYgBAZgBDbgBF8gBDNgBAegBAfgBA4gCAagCA7gCl-yowAbAAgHS

In [404]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7'
}
url = hotel_urls[0]
r = requests.get(url, headers=headers)

response = Selector(text=r.text)
r

<Response [200]>

In [405]:
hotel_name= response.css("h2.d2fee87262.pp-header__title::text").get()
hotel_name

'Hôtel Le Pré Galoffre'

In [406]:
hotel_address= response.css("div.a53cbfa6de.f17adf7576::text").get()
hotel_address

'3115 Route De Générac, 30900 Nîmes, France'

In [407]:
hotel_description= response.css("p.a53cbfa6de.b3efd73f69::text").get()
hotel_description

'L’Hôtel Le Pré Galoffre bénéficie d’un emplacement paisible dans un cadre rural à 10\xa0minutes de route du centre de Nîmes. L’établissement possède une piscine extérieure, une terrasse bien exposée et une connexion Wi-Fi gratuite.\n\nLes chambres se trouvent dans un bâtiment provençal traditionnel en pierre. Tous les hébergements disposent de la climatisation et d’une télévision à écran plat.\n\nL’Hotel Galoffre propose chaque jour un grand choix de plats pour le petit-déjeuner, et vous pourrez prendre vos repas au bord de la piscine par beau temps.\n\nLe Pré Galoffre peut assurer un service de location de vélos et de voitures. Des paniers-repas sont également disponibles pour le déjeuner. Vous aurez la possibilité de partir en randonnée et de faire de l’équitation dans les environs.\n\nUn parking est disponible gratuitement sur place si vous voyagez en voiture. Les autoroutes\xa0A9 et A54 se trouvent à 4\xa0km. L’aéroport de Nîmes-Alès-Camargue-Cévennes est à 15\xa0minutes de route.

In [408]:
hotel_rating = response.css("div.ac4a7896c7::text").get().split()[-1]
hotel_rating

'8.2'

In [409]:
latlng = response.css('a#map_trigger_header_pin::attr(data-atlas-latlng)').get()
latlng

'43.78858215364223,4.35380257666111'

In [410]:
latlng_cleaned = latlng.replace('\u200b', '').replace(' ', '').strip()
lat, lon = latlng_cleaned.split(',')
print(f"Latitude: {lat}, Longitude: {lon}")

Latitude: 43.78858215364223, Longitude: 4.35380257666111
